In [4]:
from Bio import SeqIO
import arnie
from arnie.pk_predictors import pk_predict
from arnie.utils import *
from arnie.utils import _group_into_non_conflicting_bp
import pandas as pd

def get_seq(seq_filename):
    record = SeqIO.read(seq_filename, "fasta")
    return str(record.seq)

def get_sliding_windows(full_seq, step, window):
    coords = list(range(0,len(full_seq)-window+1,step))
    windows = []
    for i in coords:
        new_window = full_seq[i:i+window]
        windows.append(new_window)
    return windows, coords

def stable_helices(dotbracket):
    bp_list = convert_dotbracket_to_bp_list(dotbracket, allow_pseudoknots=True)
    groups = _group_into_non_conflicting_bp(bp_list)
    for idx, pairs in enumerate(groups):
        if idx > 0:
            if len(pairs) == 1:
                return False 
            
##this function returns a list of all structures predicted for all windows in the genome            
def get_structures(seq_windows, coords, pk_predictor):
    struct_list = []
    for seq,coord in zip(seq_windows,coords):
        dotbracket = pk_predict(seq, pk_predictor)
        if is_PK:
            if stable_helices:
                struct_list.append([coord, coord+window, seq, dotbracket, 'Yes', 'Yes'])
            else:
                struct_list.append([coord, coord+window, seq, dotbracket, 'Yes', 'No'])
        else:
            struct_list.append([coord, coord+window, seq, dotbracket, 'No', 'nan'])
    df = pd.DataFrame(struct_list,columns=["start","end","sequence", "struct", "pseudoknot", "stable pk helix"])
    return df

#input is list of shape values as floats with nan values numpy objects 
def normalize_shape(shape_reacs):
    shape_reacs = np.array(shape_reacs)

    # Get rid of nan values for now
    nonan_shape_reacs = shape_reacs[~np.isnan(shape_reacs)]

    # Find Filter 1: 1.5 * Inter-Quartile Range
    sorted_shape = np.sort(nonan_shape_reacs)
    q1 = sorted_shape[int(0.25 * len(sorted_shape))]
    q3 = sorted_shape[int(0.75 * len(sorted_shape))]
    iq_range = abs(q3 - q1)
    filter1 = next(x for x, val in \
        enumerate(list(sorted_shape)) if val > 1.5 * iq_range)

    # Find Filter 2: 95% value
    filter2 = int(0.95 * len(sorted_shape))

    # Get maximum filter value and fiter data
    filter_cutoff = sorted_shape[max(filter1, filter2)]
    sorted_shape = sorted_shape[sorted_shape < filter_cutoff]

    # Scalefactor: Mean of top 10th percentile of values
    top90 = sorted_shape[int(0.9 * len(sorted_shape))]
    scalefactor = np.mean(sorted_shape[sorted_shape > top90])
        
    # Scale dataset
    return shape_reacs/scalefactor

# input is text file of any shape data set, output is normalized list of values in a list (some np.nan objects)
def get_normalized_shape_data(shape_filename):

    # write shape text file to list
    shape_file = open("{}".format(shape_filename), "r")
    shape_data = shape_file.read()
    shape_data_list = shape_data.split("\n")
    shape_file.close()
    
    shape_nan_list = []
    for char in shape_data_list:
        if char == '':
            shape_data_list.remove('')
        elif (char == '-999') or (char == 'nan') or (char == "NaN"):
            shape_nan_list.append('nan')
        else: 
            shape_nan_list.append(float(char))
    
    #convert string 'nan' to np.nan
    shape_reacs = []
    for char in shape_nan_list:
        if char == 'nan':
            shape_reacs.append(np.nan)
        else:
            shape_reacs.append(char)
    
    # normalize shape data
    normalized_shape_data = normalize_shape(shape_reacs).tolist()
    return normalized_shape_data

#add in this capability later
#def get_pseudoknots_with_shapeknots()

def viral_knots(seq_filename, shapeknots=False, shape_rankings=False, shape_data_folder=None, 
                shape_data_sets=None, pk_predictors, step, window):
    ### args:
        ### seq_filename - fasta file containing RNA sequence for viral genome
        ### shapeknots - if shapeknots is one of the predictors desired to use (must include shape data)
        ### shape_rankings - if shape reactivity values are used to score the pseudoknots (must include shape data)
        ### shape_data_folder - folder containing csv's with shape reactivity
        ### shape_data_sets - names of the reactivity files (no .csv necessary) 
            #(note that this is designed to work with single or multiple tracks of shape data)
        ### pk_predictors - list of desired predictors for use: threshknots, knotty, pknots, spotrna
        ### step - desired number of nucleotides to slide each window
        ### window - desired size of window to divide genome into
    
    
    ##first retrieve viral sequence and normalized shape data (if directed) and sort into windows
    seq = get_seq(seq_filename)
    RNA_seq = seq.replace("T", "U")
    seq_windows, coords = get_sliding_windows(RNA_seq, step=step, window=window)
    
    shape_sets = []
    all_shape_windows = []
    if shapeknots or shape_rankings:
        for track in shape_data_sets:
            shape_data = get_normalized_shape_data(shape_data_folder+'/'+track+'.csv')
            shape_sets.append(shape_data)
        
        for track in shape_sets:
            shape_windows, shape_coords = get_sliding_windows(track, step=step, window=window)
            all_shape_windows.append(shape_windows)
    
    ##run necessary data through each predictor and sort output into individual csvs
    ##update this function so that it saves all structures, not just pseudoknots, 
        ##but tags the structures that are pseudoknots
    
    dfs = []
    for name in pk_predictors:
        dfs.append(get_structures(seq_windows, coords, pk_predictor=name))
    
    ##run shapeknots - once implemented: figure out how to write to multiple nodes 
    
    #shapeknots_dfs = []
    #if shapeknots:
        #shapeknots_dfs.append(run_shapeknots(...))
    
    ##add the dataframe for shapeknots results to the dataframe for all other predictors
        ##so that all predictions are in the same dataframe 
        
    ##run consensus analysis and generate scores for each individual structure
    ##perform calculations to determine outliers and mark them 
    ##determine the average consensus for all structures in that window, with the exception of outliers
    ##repeat this process but focusing only on base pairs involved in pseudoknots
    
    ##determine the shape agreement of each individual structure with each track of shape data available
    ##determine if there are any outliers and mark them 
    ##find the average shape score for structures within a given window, with the exception of outliers
    ##repeat this process for all separate tracks of shape data
    ##determine if the scores from any tracks of shape data differ from each other greatly - save all individually
    ##calculate an average score for shape agreement for each window across all tracks of data
    ##repeat this process, but focusing only on base pairs involved in pseudoknots
    
    ##reorder data into a new dataframe which contains only windows that have been tagged to contain pseudoknots 
    ##each row contains the window, the sequence, a dictionary of all predicted structures(including non-pseudoknots)
        ##the average consensus score, average shape scores, and whether or not helices are stable (have at least 2 bps)
    
    ##impose thresholds for consensus and shape (both for the whole structure and for the pseudoknotted base pairs only)
    ##filter for only those with stable helices